In [1]:
%reload_ext autoreload
%autoreload 2

import sys
import os

path_to_project = os.path.abspath(os.path.join(os.getcwd(), '../'))    
sys.path.insert(1, os.path.join(path_to_project))

In [2]:
import itertools
import pandas as pd
from sklearn.preprocessing import StandardScaler
from src.utils import one_hot
from src.directory import data_dir, NHANES_preprocessed_filename
from src.estimators import aipw_estimator, unadjusted_DM_estimator, ipw_estimator, t_learner, s_learner, x_learner

In [3]:
# load df
NHANES_preprocessed_filepath = os.path.join(data_dir, NHANES_preprocessed_filename)
df = pd.read_csv(NHANES_preprocessed_filepath, index_col='SEQN')

In [4]:
# define relevant features
# features
z_col = 'ambient_light'
t_col = 'sleep_deprivation'
y_cols = ['HTN', 'DBP', 'SBP']

# column types
categorical_cols = ['physical_activity', 'depression', 'ANTIDEPRESSANTS_ANXIOLYTICS', 'GLUCOCORTICOIDS', 'sleep_troubles',
       'sleep_deprivation', 'diabetes', 'smoker', 'race_ethnicity', 'gender', 'HTN']
numerical_cols = ['daily_sedentary', 'accelerometer', 'BMI', 'age', 'poverty_ratio', 'DBP', 'SBP']

# columns not to transform
all_cols = df.columns
untransformed_cols = [x for x in all_cols if x in [*y_cols, t_col, z_col]]

In [5]:
# prune df
df.dropna(how='any', inplace=True)

## transform df
# apply scalers
scaler = StandardScaler()
numerical_transformation_cols = list(set(numerical_cols) - set(untransformed_cols))
df[numerical_transformation_cols] = scaler.fit_transform(df[numerical_transformation_cols])

# make z col binary (indicator of recommended max lux value during sleep)
light_cutoff = 1 # nightly minute-mean summed light exposure
df[z_col] = df[z_col].apply(lambda x: 1 if x <= light_cutoff else x)
df[z_col] = df[z_col].apply(lambda x: 0 if x > light_cutoff else x)

# one-hot encode multiclass categoricals
multiclass_cols = df[categorical_cols].columns[df[categorical_cols].nunique() > 2].tolist() 
categorical_transformation_cols = list(set(multiclass_cols) - set(untransformed_cols))
df = one_hot(df, categorical_transformation_cols)
df.columns = df.columns.str.replace('.0', '')

# get df as float
df = df.astype(float)

# get covariates
x_cols = list(set(df.columns) - set([t_col, *y_cols, z_col]))

# update lists of variable type
all_cols = df.columns
categorical_cols = [x for x in df.columns if any([x.startswith(y) for y in categorical_cols])]

In [6]:
estimators = [unadjusted_DM_estimator, ipw_estimator, t_learner, s_learner, x_learner, aipw_estimator]
estimator_names = estimator_names = [x.__name__ for x in estimators]
pairs = [' vs '.join(x[::-1]) for x in itertools.combinations(df[t_col].unique().astype(int).astype(str), 2)]
index = pd.MultiIndex.from_product([estimator_names, pairs, y_cols], names=['estimator', 'pair','outcome'])
tau_results = pd.DataFrame(index=index, columns=['tau'])

In [7]:
for tau_estimator, outcome in itertools.product(estimators, y_cols):
    estimator_name = tau_estimator.__name__
    print(f'Estimating effect of {t_col} on {outcome} using {estimator_name}...')
    continuous_outcome = False if outcome == 'HTN' else True
    results = tau_estimator(df, 
                            treatment_var=t_col, 
                            outcome_var=outcome, 
                            covariates=x_cols,
                            continuous_outcome=continuous_outcome)
    
    for pair in results.keys():
        tau = results[pair]['tau']
        tau_results.loc[(estimator_name, pair, outcome)] = tau

Estimating effect of sleep_deprivation on HTN using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on DBP using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on SBP using unadjusted_DM_estimator...
Estimating effect of sleep_deprivation on HTN using ipw_estimator...
Estimating effect of sleep_deprivation on DBP using ipw_estimator...
Estimating effect of sleep_deprivation on SBP using ipw_estimator...
Estimating effect of sleep_deprivation on HTN using t_learner...
Estimating effect of sleep_deprivation on DBP using t_learner...
Estimating effect of sleep_deprivation on SBP using t_learner...
Estimating effect of sleep_deprivation on HTN using s_learner...
Estimating effect of sleep_deprivation on DBP using s_learner...
Estimating effect of sleep_deprivation on SBP using s_learner...
Estimating effect of sleep_deprivation on HTN using x_learner...
Estimating effect of sleep_deprivation on DBP using x_learner...
Estimating effect of sleep_deprivati

In [8]:
tau_results

tau
estimator               pair   outcome           
unadjusted_DM_estimator 1 vs 0 HTN      -0.033383
                               DBP       0.639493
                               SBP      -1.638785
                        2 vs 0 HTN       0.037845
                               DBP       1.420203
                               SBP       0.265897
                        2 vs 1 HTN       0.071228
                               DBP        0.78071
                               SBP       1.904682
ipw_estimator           1 vs 0 HTN       0.010542
                               DBP       1.974479
                               SBP       3.337662
                        2 vs 0 HTN      -0.076871
                               DBP     -10.994843
                               SBP     -19.552696
                        2 vs 1 HTN      -0.087412
                               DBP     -12.969322
                               SBP     -22.890358
t_learner               1 vs 0 HTN            0.0
                               DBP            0.0
                               SBP            0.0
                        2 vs 0 HTN            0.0
                               DBP            0.0
                               SBP            0.0
                        2 vs 1 HTN            0.0
                               DBP            0.0
                               SBP            0.0
s_learner               1 vs 0 HTN      -0.004149
                               DBP      -0.203821
                               SBP      -0.219357
                        2 vs 0 HTN       0.005453
                               DBP      -0.305916
                               SBP      -0.146955
                        2 vs 1 HTN       0.009602
                               DBP      -0.102096
                               SBP       0.072401
x_learner               1 vs 0 HTN      -0.006571
                               DBP      -0.389846
                               SBP      -0.469339
                        2 vs 0 HTN       0.004915
                               DBP      -0.328549
                               SBP      -0.073401
                        2 vs 1 HTN       0.011486
                               DBP       0.061297
                               SBP       0.395938
aipw_estimator          1 vs 0 HTN      -0.009238
                               DBP      -0.550984
                               SBP      -0.694641
                        2 vs 0 HTN      -0.004854
                               DBP      -0.703842
                               SBP      -0.094263
                        2 vs 1 HTN       0.004384
                               DBP      -0.152858
                               SBP       0.600378